# Tutorial 1: SFT with LoRA

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_a_LLM_LIMA_CPU.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

The upcoming section will demonstrate an example of fine-tuning a model using the LoRA approach on a small dataset. This technique allows for instruction tuning of large language models by applying low-rank adaptations to modify the model’s behavior without retraining its weights. This process is highly efficient and can be executed using a CPU on a Google Cloud instance. We will guide you through preparing the dataset and conducting the fine-tuning process using the Hugging Face library. This hands-on example will provide a practical understanding of enhancing model performance with minimal computational resources and dataset size.

While the Hugging Face library is used in this example, the following libraries provide a range of tools, optimizations, compatibility with various data types, resource efficiency, and user-friendly interfaces to support different tasks and hardware setups, enhancing the efficiency of the LLM fine-tuning process.

-   [PEFT Library](https://github.com/huggingface/peft): The Parameter-Efficient Fine-Tuning (PEFT) library enables the efficient adaptation of pre-trained language models to various downstream applications without fine-tuning all the model’s parameters. Methods like LoRA, Prefix Tuning, and P-tuning are part of PEFT.
-   [Lit-GPT](https://github.com/Lightning-AI/lit-gpt): Developed by LightningAI, Lit-GPT is also an open-source tool designed to streamline fine-tuning. It facilitates the application of techniques like LoRA without manual modifications to the core model architecture. Models such as  [Vicuna](https://lmsys.org/blog/2023-03-30-vicuna/),  [Pythia](https://www.eleuther.ai/papers-blog/pythia-a-suite-for-analyzing-large-language-modelsacross-training-and-scaling), and  [Falcon](https://falconllm.tii.ae/)  are available. Lit-GPT allows applying specific configurations to different weight matrices and offers adjustable precision settings to manage memory usage effectively.

We also recommend using virtual machines on systems like GCP, which is good practice in the industry and might be better than using your current system in most cases. To do so, log in to the Google Cloud Platform account and create a  [Compute Engine](https://cloud.google.com/compute)  instance. You can select from various  [machine types](https://cloud.google.com/compute/docs/cpu-platforms). Cloud GPUs are a popular option for many deep learning applications, but CPUs can also effectively optimize LLMs. For this example, we train the model using a CPU.

Whatever type of machine you choose to use, if you encounter an out-of-memory error, try reducing parameters such as `batch_size or seq_length`.

>⚠️**Note:**  Costs are associated with starting up virtual machines. The total cost will depend on the type of machine and how long it is running. Regularly check your costs in the billing section of GCP and turn off your virtual machines when you’re not using them.

>💡If you want to run the code in the section without spending much money, you can perform a few iterations of training on your virtual machine and then stop it.

Install the dependences:

    !pip install -q transformers==4.32.0 deeplake[enterprise]==3.7.1 trl==0.6.0 peft==0.5.0 wandb==0.15.8


### Load API Keys

In [ ]:
from fine_tuning_custom_utils.helper import get_openai_api_key, get_activeloop_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()

## Loading the Dataset

The quality of a model’s output is directly tied to the quality of the data used for training. Start with a well-planned dataset, whether open-source or custom-created. We will use the dataset from the “[LIMA: Less Is More for Alignment](https://arxiv.org/pdf/2305.11206.pdf)” paper. This research suggests that a small, meticulously selected dataset of a thousand samples could replace the RLHF strategy. This research is publicly available under a non-commercial use license.

The Deep Lake database infrastructure by Activeloop enables dataset streaming, eliminating the need to download and load the dataset into memory.

The code will create a loader object for the training and test sets:

In [ ]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/GAIR-lima-train-set')
ds_test = deeplake.load('hub://genai360/GAIR-lima-test-set')

print(ds)

    Dataset(path='hub://genai360/GAIR-lima-train-set', read_only=True, tensors=['answer', 'question', 'source'])

The pre-trained tokenizer object for the [Open Pre-trained Transformer (OPT)](https://arxiv.org/abs/2205.01068) LLM is initially loaded using the transformers library, and the model is loaded later. We chose OPT for its open availability and relatively moderate parameter count. However, the code in this section is versatile and can be applied to other models. For example, you could use `meta-llama/Llama-2-7b-chat-hf` for [Llama 2](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

The `prepare_sample_text processes` a row of data stored in Deep Lake, organizing it to start with a question and follow with an answer, separated by two newlines. Defining a formatting function helps the model learn the template and recognize that a prompt beginning with the `question` keyword should typically be completed with an answer.

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"""Question: {example['question'].text()}\n\nAnswer: {example['answer'].text()}"""

    return text

The Hugging Face  [TRL library](https://github.com/huggingface/trl)  integrates the SFT method and allows the integration of LoRA configurations, simplifying the implementation.

To prepare the training and evaluation datasets for fine-tuning, we use the `ConstantLengthDataset` object from the TLR library. This object requires a tokenizer, the Deep Lake dataset, and a formatting function, `prepare_sample_text`, which processes the data into the required format.

The `seq_length` parameter determines the maximum sequence length used during training and evaluation, aligning it with the model’s configuration. While the value can be set as high as 2048 for models that can handle long contexts, we’ve selected 1024 in this example to optimize memory usage. You may increase this value depending on your dataset’s text length and available resources.

Additionally, setting `infinite=True` ensures that once all data points are exhausted, the dataset iterator automatically restarts, making it ideal for continual training scenarios.

In [ ]:
from trl.trainer import ConstantLengthDataset

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=1024
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_test,
    formatting_func=prepare_sample_text,
    seq_length=1024
)

# Show one sample from train set
iterator = iter(train_dataset)
sample = next(iterator)
print(sample)

    {'input_ids': tensor([ 2, 45641, 35, ..., 48443, 2517, 742]), 'labels': tensor([ 2, 45641, 35, ..., 48443, 2517, 742])}

The output shows that the `ConstantLengthDataset` class took care of all the necessary steps to prepare our dataset.

>💡If you use the iterator to print a sample from the dataset, execute the following code to reset the iterator pointer: train_dataset.start_iteration = 0.